<a href="https://colab.research.google.com/github/ReAlex1902/public/blob/main/test_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pandasql
# !apt-get update
# !apt-get upgrade sqlite3

# !curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
# %cd sqlite/
# !./configure
# !make sqlite3.c
# %cd /content
# !npx degit coleifer/pysqlite3 -f
# !cp sqlite/sqlite3.[ch] .
# !python setup.py build_static build
# !cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
#      /usr/lib/python3.7/lib-dynload/

In [ ]:
import numpy as np
import pandas as pd
from pandasql import sqldf

import sqlite3
sqlite3.sqlite_version

# Creating dataset
imagine we have bank payments dataset of buying and selling products with such features:
- user_id *(integer)* - unique ID of the user
- product_id *(integer)* - unique ID of the product
- transact_id *(integer)* - unique ID for each transaction
- direction *(string)* - binary, may be "incoming" or "outgoing"
- amount *(float)* - amount of money transferred
- time *(datetime)* - time of transaction
- date *(date)* - date of transaction
- purpose *(string)* - purpose of the transaction

In [ ]:
def generate_purpose(n):
    purchase = 0.5
    rent = 0.3
    service = 0.1
    non_commercial = 0.07
    tax = 0.03
    
    purposes = ['purchase', 'rent', 'service', 'non_commercial', 'tax']

    return np.random.choice(purposes, size = n, p = [0.5, 0.3, 0.1, 0.07, 0.03])


n = 10000                                                              ## number of rows in the dataset
sigma = 1000

unique_user_ids = np.arange(1, 1001)                                   ## 1000 unique user ids
unique_product_ids = np.arange(1, 101)                                 ## 100 unique product ids
unique_directions = ['incoming', 'outgoing']
amounts = sigma * np.random.lognormal(size=n)

user_ids    = np.random.choice(unique_user_ids, n)
product_ids = np.random.choice(unique_product_ids, n)
directions = np.random.choice(unique_directions, n)
transact_id = np.arange(1, n+1)

start_date = pd.to_datetime('2016-01-01')
times = pd.date_range(start_date, periods=n, freq='300min')       ## let's have actions begin at March 1st 2022 with 5 min gaps

bank_payments = pd.DataFrame({'user_id': user_ids, 
                              'product_id': product_ids, 
                              'transact_id': transact_id,
                              'direction': directions,
                              'amount': amounts,
                              'time': times})

bank_payments['date'] = bank_payments['time'].dt.date
bank_payments['purpose'] = generate_purpose(n)
bank_payments

,user_id,product_id,transact_id,direction,amount,time,date,purpose
0,874,92,1,incoming,579.252327,2016-01-01 00:00:00,2016-01-01,service
1,843,78,2,incoming,1473.619265,2016-01-01 05:00:00,2016-01-01,service
2,15,44,3,outgoing,2342.919365,2016-01-01 10:00:00,2016-01-01,rent
3,305,77,4,incoming,1873.132968,2016-01-01 15:00:00,2016-01-01,non_commercial
4,487,69,5,outgoing,2376.197751,2016-01-01 20:00:00,2016-01-01,purchase
...,...,...,...,...,...,...,...,...
9995,238,78,9996,incoming,332.971034,2021-09-13 07:00:00,2021-09-13,rent
9996,584,30,9997,outgoing,498.545080,2021-09-13 12:00:00,2021-09-13,purchase
9997,24,11,9998,outgoing,1388.937854,2021-09-13 17:00:00,2021-09-13,purchase
9998,69,48,9999,incoming,1216.225898,2021-09-13 22:00:00,2021-09-13,purchase


# Scripts
1. Count average and sum of amounts for incoming non-commercial purposes 

In [ ]:
query = '''
    select
        product_id,
        avg(amount) mean_amount_incoming,
        sum(amount) sum_amount_incoming
    from
        bank_payments
    where
        purpose = 'non_commercial'
        and direction = 'incoming'
    group by
        product_id
    order by
        sum_amount_incoming desc
'''

sqldf(query)

,product_id,mean_amount_incoming,sum_amount_incoming
0,28,2704.404813,18930.833690
1,13,5486.139487,16458.418462
2,17,3281.586977,16407.934886
3,51,4778.547688,14335.643064
4,63,2564.835156,12824.175779
...,...,...,...
91,15,757.105807,757.105807
92,56,556.260912,556.260912
93,10,233.623463,467.246927
94,59,341.473081,341.473081


2. Count sum of incoming purchase amounts from the beginning of 2018 to the end of 2020

In [ ]:
query = '''
    select
        user_id,
        transact_id,
        amount,
        sum(amount) over(partition by user_id order by time) sum_amount
    from
        bank_payments
    where
        purpose = 'purchase'
        and direction = 'incoming'
        and date >= '2018-01-01'
        and date <= '2020-12-31'
    group by
        product_id
    order by
        user_id
'''

sqldf(query)

,user_id,transact_id,amount,sum_amount
0,3,4388,295.985425,295.985425
1,10,3531,1536.450174,1536.450174
2,14,4209,1609.573156,1609.573156
3,36,3546,3204.788734,3204.788734
4,57,3583,5596.429621,5596.429621
...,...,...,...,...
95,955,3836,1045.368583,1045.368583
96,955,4361,490.853104,1536.221687
97,963,4122,1410.395369,1410.395369
98,987,3573,1678.976490,1678.976490


3. Incoming and outgoing transact activity for existing purposes

In [ ]:
query = '''
    select
        user_id,
        sum(case when purpose = 'purchase'       and direction = 'incoming' then amount else 0 end) purchase_incoming,
        sum(case when purpose = 'rent'           and direction = 'incoming' then amount else 0 end) rent_incoming,
        sum(case when purpose = 'service'        and direction = 'incoming' then amount else 0 end) service_incoming,
        sum(case when purpose = 'non_commercial' and direction = 'incoming' then amount else 0 end) non_commercial_incoming,
        sum(case when purpose = 'tax'            and direction = 'incoming' then amount else 0 end) tax_incoming,

        sum(case when purpose = 'purchase'       and direction = 'outgoing' then amount else 0 end) purchase_outgoing,
        sum(case when purpose = 'rent'           and direction = 'outgoing' then amount else 0 end) rent_outgoing,
        sum(case when purpose = 'service'        and direction = 'outgoing' then amount else 0 end) service_outgoing,
        sum(case when purpose = 'non_commercial' and direction = 'outgoing' then amount else 0 end) non_commercial_outgoing,
        sum(case when purpose = 'tax'            and direction = 'outgoing' then amount else 0 end) tax_outgoing
    from
        bank_payments
    where
        date >= '2020-01-01'
        and date <= '2020-12-31'
    group by
        user_id
    order by
        user_id
'''

sqldf(query)

,user_id,purchase_incoming,rent_incoming,service_incoming,non_commercial_incoming,tax_incoming,purchase_outgoing,rent_outgoing,service_outgoing,non_commercial_outgoing,tax_outgoing
0,1,616.396412,1566.084713,0.000000,0.0,0.000000,2782.797137,4440.084647,0.000000,0.000000,0.000000
1,2,0.000000,1707.084942,0.000000,0.0,0.000000,3004.357069,377.703394,0.000000,0.000000,0.000000
2,3,3183.717320,0.000000,0.000000,0.0,0.000000,0.000000,3506.833914,1272.671205,1952.906657,0.000000
3,4,576.995937,2361.809994,0.000000,0.0,0.000000,613.132157,7380.265076,0.000000,0.000000,0.000000
4,5,1688.369204,174.868731,0.000000,0.0,0.000000,1841.448557,590.146410,3687.407578,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
988,996,0.000000,265.984426,866.389434,0.0,0.000000,812.018565,1952.918959,0.000000,0.000000,145.924789
989,997,0.000000,0.000000,705.828061,0.0,0.000000,1313.067220,7223.336067,0.000000,0.000000,0.000000
990,998,88.836420,0.000000,0.000000,0.0,1957.997322,2168.978948,4441.227996,0.000000,0.000000,0.000000
991,999,1352.014269,677.769478,0.000000,0.0,1150.850282,4127.651298,396.407476,1256.638725,0.000000,0.000000
